In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_17633/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_17633/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_42_6,0.935743,0.794004,0.820777,0.875214,0.027531,0.122288,0.125549,0.053242,0.068542,0.165924,0.925990,0.167216,41.184892,89.932978,"Hidden Size=[4], regularizer=0.02, learning_ra..."
1,model_1_42_2,0.935743,0.794004,0.820777,0.875214,0.027531,0.122288,0.125549,0.053242,0.068542,0.165924,0.925990,0.167216,41.184892,89.932978,"Hidden Size=[4], regularizer=0.02, learning_ra..."
2,model_1_42_9,0.935743,0.794004,0.820777,0.875214,0.027531,0.122288,0.125549,0.053242,0.068542,0.165924,0.925990,0.167216,41.184892,89.932978,"Hidden Size=[4], regularizer=0.02, learning_ra..."
3,model_1_43_2,0.936446,0.793811,0.821963,0.951806,0.027230,0.122403,0.124719,0.017389,0.068542,0.165014,0.926800,0.166298,41.206906,89.954992,"Hidden Size=[4], regularizer=0.02, learning_ra..."
4,model_1_44_7,0.937798,0.793594,0.822587,0.960265,0.026650,0.122531,0.124281,0.023915,0.068923,0.163249,0.928357,0.164520,41.249908,89.997994,"Hidden Size=[4], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492,model_2_0_0,0.108279,-0.121668,-0.123976,0.247514,0.382058,0.665871,0.787368,0.301362,0.343376,0.618109,-0.198251,0.622919,67.924365,162.553002,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1493,model_5_0_0,0.038633,-0.145577,-0.188185,-0.282886,0.411898,0.680064,0.832349,0.530196,0.361875,0.641792,8.751017,0.646787,291.773961,707.566456,"Hidden Size=[36], regularizer=0.02, learning_r..."
1494,model_8_0_0,-0.151847,-0.167238,-0.310897,-0.174582,0.493509,0.692923,0.918311,0.521296,0.376823,0.702502,-17.573537,0.707969,243.412428,590.384097,"Hidden Size=[12, 6], regularizer=0.02, learnin..."
1495,model_8_0_3,0.169313,-0.247332,-0.309223,0.178512,0.355908,0.740470,0.917138,0.364588,0.325009,0.596580,-12.394834,0.601223,244.066165,591.037834,"Hidden Size=[12, 6], regularizer=0.02, learnin..."
